# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1631937024592_0301'

In [3]:
pd.options.display.max_columns = 35

In [4]:
from foresight.discern import broadcast_discern, push_discern, pop_discern

### care management new default will be 5e259fd5-75b5-4d49-82d3-2d4e92dca831 for any new clients or 
###  any algorithm wiki not line for line with Cerner Standard then would be using the new context  

pd.set_option('display.max_colwidth', None)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'   ##<------------- change it to your stack without the v1/

contextid = ['da0acee6-1e2e-4384-9fc7-a58c259d0c50', '5E259FD575B54D4982D32D4E92DCA831'] 

# Loop that removes all hypthens and captializes all letters. Sets up the file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    print(contextid)

defaultcontext = 'DA0ACEE61E2E43849FC7A58C259D0C50'   
defaultcontext1 = '5E259FD575B54D4982D32D4E92DCA831' 

DA0ACEE61E2E43849FC7A58C259D0C50
5E259FD575B54D4982D32D4E92DCA831


In [5]:
db = "real_world_data_2021_Q2"
spark.sql("USE {}".format(db))
print("Using database: {},".format(db))

Using database: real_world_data_2021_Q2,


In [6]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,real_world_data_2021_q2,allergy,False
1,real_world_data_2021_q2,clinical_event,False
2,real_world_data_2021_q2,condition,False
3,real_world_data_2021_q2,demographics,False
4,real_world_data_2021_q2,encounter,False
5,real_world_data_2021_q2,immunization,False
6,real_world_data_2021_q2,lab,False
7,real_world_data_2021_q2,measurement,False
8,real_world_data_2021_q2,medication,False
9,real_world_data_2021_q2,medication_administration,False


### Create Variables for Queries

In [7]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [8]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS {userName}""".format(**{
    'userName':userName
})).show()

++
||
++
++



In [9]:
spark.sql(f"""select * FROM {userName}.{algorithmName}_Hospital h  """).printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- tenant_bed_size: string (nullable = true)
 |-- tenant_speciality: string (nullable = true)
 |-- tenant_segment: string (nullable = true)
 |-- tenant_zip_code: string (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- dischargeDate: string (nullable = true)
 |-- length_of_stay: integer (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- deceased: integer (nullable = true)
 |-- discharged_to_skilled_nursing_facility: integer (nullable = true)
 |-- discharged_to_hospice: integer (nullable = true)
 |-- died_in_the_hospital: integer (nullable = true)
 |-- discharged_to_rehab_facility: integer (nullable = true)
 |-- discharged_to_long_term_care_facility: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- gender: string (nullable = true)



In [10]:
spark.sql("""select * from procedure """).printSchema()

root
 |-- procedureid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- procedurecode: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- modifiercodes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- standard: struct (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |    |    |-- standardCodings: arr

In [11]:
procedureDf = spark.sql(f"""


    SELECT distinct 
        h.tenant
        ,h.personId
        ,h.encounterId
        ,p.procedureId
        ,if(HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
                            array('MECHANICAL_VENTILATION_PROC'),
                            '{defaultcontext}'),1,0) as on_mechanical_ventilation
 --       ,if(HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
 --                           array('MECHANICAL_VENTILATION_PROC'),
 --                           '{defaultcontext}'), if(length(serviceEndDate) > 3,
 --                                    datediff(p.serviceEndDate, p.serviceStartDate)+1, 
 --                                    datediff(h.dischargeDate, p.serviceStartDate)+1), 0) 
 --                           as number_of_days_on_mechanical_ventilation                  
        ,if(HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
                            array('CHEST_CT_SCAN_PROC'),
                            '{defaultcontext}'),1,0) as had_chest_ct_scan    
        ,if(HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
                            array('CHEST_MRI_PROC'),
                            '{defaultcontext}'),1,0) as had_chest_mri 
        ,if(HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
                            array('CHEST_MRI_PROC'),
                            '{defaultcontext}'),1,0) as had_chest_ultrasound  
        ,if(HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
                            array('CHEST_XRAY_PROC'),
                            '{defaultcontext}'),1,0) as had_chest_xray  
                   
        ,p.serviceStartDate
        ,p.serviceEndDate



  
        FROM {userName}.{algorithmName}_Hospital h  

       

        JOIN {db}.procedure p
            ON  h.personId = p.personId
                and h.tenant = p.tenant
                and h.encounterId = p.encounterId
                AND HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
                            array('MECHANICAL_VENTILATION_PROC','CHEST_CT_SCAN_PROC', 'CHEST_MRI_PROC',
                                  'CHEST_ULTRASOUND_PROC', 'CHEST_XRAY_PROC'),
                               '{defaultcontext}' ) 
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(p.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                            
      
                       
   
   
""")

In [12]:
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_HospitalProcedure""").show() 

++
||
++
++



In [13]:
procedureDf.write.saveAsTable(f"""{userName}.{algorithmName}_HospitalProcedure""")

In [14]:
procedureDf.count()

18084

In [15]:
spark.sql(f"""select p.*
                    ,tenant_bed_size
                    ,tenant_speciality
                    ,tenant_segment
                    ,tenant_zip_code
                    ,serviceDate
                    ,dischargeDate
                    ,length_of_stay
                    ,zip_code
                    ,deceased
                    ,discharged_to_skilled_nursing_facility
                    ,discharged_to_hospice
                    ,died_in_the_hospital 
                    ,discharged_to_rehab_facility
                    ,discharged_to_long_term_care_facility
                    ,age
                    ,gender
            from {userName}.{algorithmName}_HospitalProcedure p
            join {userName}.{algorithmName}_Hospital h
            where p.tenant = h.tenant
              and p.personId = h.personid
              and p.encounterId = h.encounterId
    --    where length(serviceEndDate) > 3 
    --     and datediff(p.serviceEndDate, p.serviceStartDate)+1 = 1
          order by 1,2,3,4
        limit(25)
""").toPandas()

,tenant,personId,encounterId,procedureId,on_mechanical_ventilation,had_chest_ct_scan,had_chest_mri,had_chest_ultrasound,had_chest_xray,serviceStartDate,serviceEndDate,tenant_bed_size,tenant_speciality,tenant_segment,tenant_zip_code,serviceDate,dischargeDate,length_of_stay,zip_code,deceased,discharged_to_skilled_nursing_facility,discharged_to_hospice,died_in_the_hospital,discharged_to_rehab_facility,discharged_to_long_term_care_facility,age,gender
0,1,01b2976d-d404-4b8d-966e-cfe99c6e3c09,b8c5dfc4-cd68-4091-bbeb-06ff7c7eda44,1617ac27-e07e-44b2-9f36-9f0cad4b953f,1,0,0,0,0,2021-01-19T07:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2021-01-06T10:56:32+00:00,2021-01-25T22:25:00+00:00,20,5,1,0,0,1,0,0,39.0,female
1,1,01b2976d-d404-4b8d-966e-cfe99c6e3c09,b8c5dfc4-cd68-4091-bbeb-06ff7c7eda44,cfd9d5f3-78d8-49dc-b8be-6c1f647b82c6,1,0,0,0,0,2021-01-08T07:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2021-01-06T10:56:32+00:00,2021-01-25T22:25:00+00:00,20,5,1,0,0,1,0,0,39.0,female
2,1,024f8035-10e5-4bb2-9a39-c4fda12eb729,6a675b01-86af-40eb-abb1-1e375cb4906b,e6b0fefe-b53d-4d86-9ec5-10395d56e615,1,0,0,0,0,2020-08-21T06:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2020-08-09T11:03:39+00:00,2020-09-29T14:34:29+00:00,52,5,1,0,0,1,0,0,69.0,male
3,1,024f8035-10e5-4bb2-9a39-c4fda12eb729,6a675b01-86af-40eb-abb1-1e375cb4906b,e6b0fefe-b53d-4d86-9ec5-10395d56e615,1,0,0,0,0,2020-08-21T06:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2020-08-09T11:03:39+00:00,2020-09-29T14:34:29+00:00,52,5,0,0,0,1,0,0,69.0,male
4,1,02565cd3-990f-48a9-a1c8-03100392fb38,df11cb43-006e-4d0a-98d9-86b91a238580,4385ecec-c59d-4b94-9526-1a5ff437be87,1,0,0,0,0,2020-05-12T06:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2020-05-12T16:24:06+00:00,2020-05-30T00:24:43+00:00,19,5,0,0,0,0,0,0,68.0,male
5,1,027acb52-785a-4c84-b6f7-4508e4a52531,ac70b3b7-61d5-4732-a8fe-3bd33215a3f1,fa353d59-c7a1-4f42-8fd0-007b667524ca,1,0,0,0,0,2020-11-15T07:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2020-11-08T21:56:59+00:00,2020-11-21T20:55:00+00:00,14,5,1,0,0,1,0,0,87.0,male
6,1,07c5a862-7680-41cf-85a3-e4a861e90790,2a6b1552-d09f-41d5-9559-c2afd7e0bf12,9b391672-8094-4615-8b41-491298b087f5,1,0,0,0,0,2020-12-15T07:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2020-12-08T15:12:22+00:00,2021-01-06T22:25:00+00:00,30,5,0,0,0,1,0,0,83.0,female
7,1,07c5a862-7680-41cf-85a3-e4a861e90790,2a6b1552-d09f-41d5-9559-c2afd7e0bf12,9b391672-8094-4615-8b41-491298b087f5,1,0,0,0,0,2020-12-15T07:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2020-12-08T15:12:22+00:00,2021-01-06T22:25:00+00:00,30,5,1,0,0,1,0,0,83.0,female
8,1,08bc997c-f4d9-4fc1-b80d-4a7df46c9218,cb2e19fd-4b8d-49ea-a46f-ead39ae31bae,32538f32-40be-48e0-a272-4c4ee90c784a,0,0,0,0,1,2020-09-02T06:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2020-09-01T22:34:00+00:00,2020-09-03T17:28:00+00:00,3,5,0,0,0,0,0,0,77.0,female
9,1,08bc997c-f4d9-4fc1-b80d-4a7df46c9218,cb2e19fd-4b8d-49ea-a46f-ead39ae31bae,afc61653-4606-4a4e-ab7c-c6b630499b8c,0,0,0,0,1,2020-09-02T06:00:00+00:00,,500-999,IDN/Regional Health Authority,Regional Hospital,5,2020-09-01T22:34:00+00:00,2020-09-03T17:28:00+00:00,3,5,0,0,0,0,0,0,77.0,female
